In [51]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler
from kmodes.kprototypes import KPrototypes

In [52]:
data = pd.read_csv('C:/Users/atirkey/OneDrive - Kmart Australia Limited/Downloads/Control.csv')

In [81]:
state_mapping = {
    "W.A": 0, "N.T": 1, "S.A": 2, "QLD": 3, "NSW": 4,
    "VIC": 5, "ACT": 6, "TAS": 7, "NTH": 8, "STH": 9, "undefined": 10
}

data["STATE"] = data["STATE"].map(state_mapping)
# data["STATE"].fillna(state_mapping["undefined"], inplace=True)
data.fillna({"STATE": state_mapping["undefined"]}, inplace=True)

In [ ]:
categorical_columns = ["STORE_TYPE", "AREA", "COUNTRY"]
categorical_mappings = {}

In [84]:
for col in categorical_columns:
    unique_vals = data[col].unique()
    col_mapping = {val : idx for idx, val in enumerate(unique_vals)}
    categorical_mappings[col] = col_mapping
    data[col] = data[col].map(col_mapping)

In [85]:
numerical_columns = ["AVERAGE_BASKET_SIZE", "NO_OF_TRANSACTIONS", "AVG_BASKET_DOLLARS", "TOTAL_SALES_UNITS", "TOTAL_SALES_DOLLARS"]
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

In [86]:
X = data[numerical_columns + ["STATE"] + categorical_columns].values

In [87]:
print(data.isnull().sum())

STORE                    0
STATE                  361
STORE_TYPE               0
AREA                     0
COUNTRY                  0
AVERAGE_BASKET_SIZE      0
NO_OF_TRANSACTIONS       0
AVG_BASKET_DOLLARS       0
TOTAL_SALES_UNITS        0
TOTAL_SALES_DOLLARS      0
REGION                   0
dtype: int64


In [70]:
kproto = KPrototypes(n_clusters = 4, init='Huang', random_state=42)
categorical_indices = [numerical_columns.index("TOTAL_SALES_DOLLARS") + i + 1 for i in range(len(categorical_columns) + 1)]
clusters = kproto.fit_predict(X, categorical=categorical_indices)

ValueError: Input contains NaN.

In [47]:
data['Cluster'] = clusters

In [48]:
print("Categorical Value Mappings:")
for col, mapping in categorical_mappings.items():
    print(f"{col} : {mapping}")

Categorical Value Mappings:
STORE_TYPE : {'KMART': 0, 'K-HUB': 1, 'undefined': 2}
AREA : {'Satellite City/Regional City': 0, 'Middle Metro': 1, 'Remote': 2, 'Inner Regional': 3, 'Outer Regional': 4, 'Inner Metro': 5, 'Outer Metro': 6, 'undefined': 7}
COUNTRY : {'AUSTRALIA': 0, 'NEW ZEALAND': 1, 'undefined': 2}


In [49]:
print("\nCluster Assignments:")
print(data[["STORE", "Cluster"]])


Cluster Assignments:
     STORE  Cluster
0     1074        0
1     1022        0
2     1275        0
3     1148        3
4     1104        0
..     ...      ...
356   3351        1
357   1111        0
358   1376        0
359   1350        0
360   1015        1

[361 rows x 2 columns]


In [50]:
data.to_csv("Prototype2.csv", index=False)